In [14]:
import pandas as pd
import numpy as np

from datetime import datetime


pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

In [44]:
def counter_nan(series):
    return sum(series.isna())


def get_percentage_missing_data(df, threshold, show_percentage=False):
    '''Inputs: Datframe, thresold
    Function: return the features with more than thresold of missing data'''
    rows=df.shape[0]
    missing_features = []
    perc_missing_features = {}
    for column in df.columns:
        counter_perc = counter_nan(df[column])/rows
        perc_missing_features[column] = round(counter_perc*100, 2)
        if counter_perc > threshold:
            missing_features.append(column)
        else:
            continue
            
    if show_percentage:
        print(perc_missing_features)
    else:
        pass
    return missing_features
# Fill nan employment status clients with indefined
def fill_nan_employment(df):
    df['employment_status_clients'] = df['employment_status_clients'].replace(to_replace=np.nan, value='non-defined')


    
    
def birthday_to_age(string_date):
    current_year = 2018
    year_birthday = int(string_date.split('-')[0])
    age = 2018-year_birthday
    return age



def get_interest_data(previous_loan_df):
    '''Input: dataframe with the previous loans data
    Function: add two new columns to the dataframe containing the interest rate and the rate amount'''
    total_due = previous_loan_df['totaldue']
    loan_amount = previous_loan_df['loanamount']
    interest = total_due-loan_amount
    interest_rate = round((interest/loan_amount)*100,2)
    previous_loan_df['interest'] = interest
    previous_loan_df['interest_rate'] = interest_rate
    
    

def scale_termdays(previous_loan_df):
    '''Scale termdays using a year as unit'''
    previous_loan_df['termdays'] = round(previous_loan_df['termdays']/365, 2)
    
    
    
def get_date_features(df, keep_customer_id=False):
    '''Return a vector with the name of the date features'''
    
    if keep_customer_id:
        date_features = ['customerid']
    else:
        date_features = []
        
    for column in df.columns:
        if 'date' in column:
            date_features.append(column)
        else:
            continue
    return date_features



def get_non_date_features(df, keep_customer_id=False):
    '''Return a vector with the non date features'''
    if keep_customer_id:
        non_date_features = ['customerid']
    else:
        non_date_features = []
        
    for column in df.columns[1:]:
        if 'date' not in column:
            non_date_features.append(column)
        else:
            continue
    return non_date_features

    

def string_to_datetime(df):
    '''Convert date with string format into date variables'''
    for date_feature in get_date_features(df):
        df[date_feature] = df[date_feature].apply(lambda date: datetime.strptime(date.split(' ')[0], '%Y-%m-%d'))
    

    
def differentiate_dates(df, columns_to_differentiate=[]):
    '''Create a new column in which we store the distance between two dates'''
    #Days scale
    name_new_column = 'diff_'+columns_to_differentiate[0]+'_'+columns_to_differentiate[1]
    sec_in_day = 60*60*24
    #
    df[name_new_column] = df[columns_to_differentiate[0]] - df[columns_to_differentiate[1]]
    df[name_new_column] = df[name_new_column].apply(lambda x: round((x.total_seconds()/sec_in_day), 2))
    
    
def date_to_day(df):
    '''Create a new variable containing in which day of the week the due and the repaid took place'''
    columns_map = {
        'firstrepaiddate': 'repaid_dayofweek',
        'firstduedate': 'due_dayofweek'
    }
    for key, value in columns_map.items():
        df[value] = df[key].apply(lambda date: datetime.weekday(date))
        
        
        
def is_month_start(df):
        columns_map = {
        'firstrepaiddate': 'is_month_start_repaid',
        'firstduedate': 'is_month_start_due'
    }
        for key, value in columns_map.items():
            df[value] = df[key].apply(lambda date: 1 if date.day == 1 else 0)
            

def get_statistics_by_customer(df):
    '''Return a df with the aggregated data by customer
    
    '''
    stats = ['max', 'min', 'mean']
    df_dict = {}
    for stat in stats:
        if stat == 'max':
            rename_feature = {}
            for column in df.columns[2:]:
                rename_feature[column] = column+'_'+stat
            df_dict[stat] = df.groupby(['customerid']).max()
            df_dict[stat] = df_dict[stat].rename(columns=rename_feature)
            df_dict[stat] = df_dict[stat].drop(columns=['loannumber'])
        elif stat == 'min':
            rename_feature = {}
            for column in df.columns[2:]:
                rename_feature[column] = column+'_'+stat
            df_dict[stat] = df.groupby(['customerid']).min()
            df_dict[stat] = df_dict[stat].rename(columns=rename_feature)
            df_dict[stat] = df_dict[stat].drop(columns=['loannumber'])
        elif stat == 'mean':
            rename_feature = {}
            for column in df.columns[2:]:
                rename_feature[column] = column+'_'+stat
            df_dict[stat] = df.groupby(['customerid']).mean()
            df_dict[stat] = df_dict[stat].rename(columns=rename_feature)
            df_dict[stat] = df_dict[stat].drop(columns=['loannumber'])
            
    df_stat = pd.concat([df_dict['max'], df_dict['min'], df_dict['mean']], axis=1)
    
    return df_stat

In [45]:
def get_final_df(demographics, prev_loans, perf, is_train_data=True):
    #Drop columns with high percentage of missing values
    columns_to_drop = get_percentage_missing_data(demographics, 0.8, show_percentage=False)
    fill_nan_employment(demographics)
    demographics = demographics.drop(columns=columns_to_drop)
    #Transform birthdate in current age of each customer
    demographics['birthdate'] = demographics['birthdate'].apply(birthday_to_age)
    #Transform prevloans
        #payment info
    get_interest_data(prev_loans)
    scale_termdays(prev_loans)
        #date features treatment
    date_features_df = prev_loans.drop(columns=get_non_date_features(prev_loans))
        #String dates to datetime format
    string_to_datetime(date_features_df)
        #Create a two features tha computes the 'speed' of repayment
    differentiate_dates(date_features_df, ['firstduedate', 'firstrepaiddate'])
    differentiate_dates(date_features_df, ['closeddate', 'creationdate'])
        #Which day of the week the due and the repaid took place
    date_to_day(date_features_df)
        #Merge the transformed date features with the prevloans data
    prev_loans = pd.concat([prev_loans,date_features_df.drop(columns=get_date_features(prev_loans, keep_customer_id=True))], axis=1)
        #Non relevant variables once transformed the data
    columns_to_drop = ['systemloanid', 'approveddate', 'creationdate', 'closeddate', 'referredby', 'firstduedate',
                  'firstrepaiddate']
    prev_loans = prev_loans.drop(columns=columns_to_drop)
        #final prev loans data
    prev_loans = get_statistics_by_customer(prev_loans)
    prev_loans['customerid'] = prev_loans.index
    prev_loans = prev_loans.reset_index(drop=True)
    
    #Merge prevloans data with demographic data
    transformed_data = demographics.merge(prev_loans, how='inner', on=['customerid'])
    final_data = perf.merge(transformed_data, how='inner', on=['customerid'])
    
    #Drop dates loans
    final_data = final_data.drop(columns=['approveddate', 'creationdate', 'referredby'])
    #one hot encoding
    final_data = pd.get_dummies(final_data, columns=['bank_account_type', 'bank_name_clients', 'employment_status_clients'])
    #Encode good_bad_flag: Bad=1 Good=0
    if is_train_data:
        final_data['good_bad_flag'] = final_data['good_bad_flag'].apply(lambda x: 1 if x=='Bad' else 0)
        final_data = final_data.drop(columns=['customerid', 'systemloanid'])
    else:
        pass
    
    return final_data

In [46]:
test_perf = pd.read_csv('/Users/ayoubelqadi/PycharmProjects/zindi_paper/data/testperf.csv', low_memory=False)
test_prevloans = pd.read_csv('/Users/ayoubelqadi/PycharmProjects/zindi_paper/data/testprevloans.csv', low_memory=False)
test_demographics = pd.read_csv('/Users/ayoubelqadi/PycharmProjects/zindi_paper/data/testdemographics.csv', low_memory=False)



In [48]:
test_data = get_final_df(demographics=test_demographics, prev_loans=test_prevloans, perf=test_perf, is_train_data=False)

In [38]:
def transform_prev_loans_dataset(test_prevloans):
    #Transform prevloans
    #payment info
    get_interest_data(test_prevloans)
    scale_termdays(test_prevloans)
    #date features treatment
    date_features_df = test_prevloans.drop(columns=get_non_date_features(test_prevloans))
    #String dates to datetime format
    string_to_datetime(date_features_df)
    #Create a two features tha computes the 'speed' of repayment
    differentiate_dates(date_features_df, ['firstduedate', 'firstrepaiddate'])
    differentiate_dates(date_features_df, ['closeddate', 'creationdate'])
    #Which day of the week the due and the repaid took place
    date_to_day(date_features_df)
    #Merge the transformed date features with the prevloans data
    prev_loans = pd.concat([test_prevloans,date_features_df.drop(columns=get_date_features(test_prevloans, keep_customer_id=True))], axis=1)
    #Non relevant variables once transformed the data
    columns_to_drop = ['systemloanid', 'approveddate', 'creationdate', 'closeddate', 'referredby', 'firstduedate',
                      'firstrepaiddate']
    test_prevloans = test_prevloans.drop(columns=columns_to_drop)
    #final prev loans data
    test_prevloans = get_statistics_by_customer(test_prevloans)
    test_prevloans['customerid'] = test_prevloans.index
    test_prevloans = test_prevloans.reset_index(drop=True)
    
    return test_prevloans

def tranform_demographics_dataset(demographics):
    #Drop columns with high percentage of missing values
    columns_to_drop = get_percentage_missing_data(demographics, 0.8, show_percentage=False)
    fill_nan_employment(demographics)
    demographics = demographics.drop(columns=columns_to_drop)
    #Transform birthdate in current age of each customer
    demographics['birthdate'] = demographics['birthdate'].apply(birthday_to_age)
    return demographics

def merge_demographics_prevloans(demographics, prev_loans):
    merged_data = demographics.merge(prev_loans, how='inner', on=['customerid'])
    return merged_data
    
    


In [39]:
test_prevloans = transform_prev_loans_dataset(test_prevloans=test_prevloans)
test_demogrphics = tranform_demographics_dataset(demographics=test_demographics)
#meged_data = merge_demographics_prevloans(test_demogrphics, prev_loans=test_prevloans)


In [49]:
test_data

,customerid,systemloanid,loannumber,loanamount,totaldue,termdays,birthdate,longitude_gps,latitude_gps,loanamount_max,totaldue_max,termdays_max,interest_max,interest_rate_max,diff_firstduedate_firstrepaiddate_max,diff_closeddate_creationdate_max,repaid_dayofweek_max,due_dayofweek_max,loanamount_min,totaldue_min,termdays_min,interest_min,interest_rate_min,diff_firstduedate_firstrepaiddate_min,diff_closeddate_creationdate_min,repaid_dayofweek_min,due_dayofweek_min,loanamount_mean,totaldue_mean,termdays_mean,interest_mean,interest_rate_mean,diff_firstduedate_firstrepaiddate_mean,diff_closeddate_creationdate_mean,repaid_dayofweek_mean,due_dayofweek_mean,bank_account_type_Current,bank_account_type_Other,bank_account_type_Savings,bank_name_clients_Access Bank,bank_name_clients_Diamond Bank,bank_name_clients_EcoBank,bank_name_clients_FCMB,bank_name_clients_Fidelity Bank,bank_name_clients_First Bank,bank_name_clients_GT Bank,bank_name_clients_Heritage Bank,bank_name_clients_Skye Bank,bank_name_clients_Stanbic IBTC,bank_name_clients_Sterling Bank,bank_name_clients_UBA,bank_name_clients_Union Bank,bank_name_clients_Unity Bank,bank_name_clients_Wema Bank,bank_name_clients_Zenith Bank,employment_status_clients_Permanent,employment_status_clients_Retired,employment_status_clients_Self-Employed,employment_status_clients_Student,employment_status_clients_Unemployed,employment_status_clients_non-defined
0,8a8589c253ace09b0153af6ba58f1f31,301982236,6,20000,24500.0,30,37,3.227945,6.586668,20000.0,23800.0,0.08,3800.0,30.00,2.0,53.0,5,2,10000.0,11500.0,0.04,1500.0,15.00,-21.0,17.0,0,0,13000.000000,15935.000000,0.072000,2935.000000,23.300000,-3.600000,32.000000,1.400000,0.600000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
1,8a858e225a28c713015a30db5c48383d,301959177,4,20000,24500.0,30,43,5.248368,13.059864,10000.0,13000.0,0.08,3000.0,30.00,6.0,26.0,5,3,10000.0,11500.0,0.04,1500.0,15.00,0.0,15.0,1,0,10000.000000,12500.000000,0.066667,2500.000000,25.000000,3.333333,22.333333,3.000000,1.666667,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
2,8a858e675c3fe0a1015c44ba8a037fee,301987980,2,10000,11500.0,15,24,5.288827,7.604053,10000.0,13000.0,0.08,3000.0,30.00,4.0,30.0,3,0,10000.0,13000.0,0.08,3000.0,30.00,4.0,30.0,3,0,10000.000000,13000.000000,0.080000,3000.000000,30.000000,4.000000,30.000000,3.000000,0.000000,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,8a858ec95afc0922015b0edc17e34984,301965287,4,20000,24500.0,30,36,3.339836,7.118107,10000.0,13000.0,0.08,3000.0,30.00,7.0,27.0,2,4,10000.0,13000.0,0.08,3000.0,30.00,4.0,23.0,0,0,10000.000000,13000.000000,0.080000,3000.000000,30.000000,5.333333,25.333333,1.333333,2.000000,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,8a858f5b5bee1b11015bf1b4ffea5abb,301981908,3,10000,11500.0,15,29,3.371619,6.605104,10000.0,13000.0,0.08,3000.0,30.00,10.0,22.0,4,3,10000.0,11500.0,0.04,1500.0,15.00,-1.0,16.0,4,0,10000.000000,12250.000000,0.060000,2250.000000,22.500000,4.500000,19.000000,4.000000,1.500000,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5,8a858fe758dc437a0159018589dc5df0,301997797,10,30000,34500.0,30,29,3.950541,7.441765,30000.0,34500.0,0.08,4500.0,30.00,11.0,34.0,4,4,10000.0,11500.0,0.04,1500.0,9.50,-3.0,6.0,0,0,20000.000000,23211.111111,0.066667,3211.111111,16.907778,1.444444,24.000000,1.888889,2.555556,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
6,8a858efd5b39cf3d015b3cb3b3882cc7,301971598,7,30000,39000.0,60,30,3.397152,6.451590,20000.0,22250.0,0.04,2250.0,15.00,3.0,17.0,3,4,10000.0,11500.0,0.04,1500.0,11.25,1.0,12.0,0,1,15000.000000,16875.000000,0.040000,1875.000000,13.125000,1.500000,14.166667,1.166667,2.666667,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
7,8a858faf56b7821c0156cdaa248222fd,301983363,3,10000,13000.0,30,28,3.300341,6.584682,10000.0,13000.0,0.08,3000.0,30.00,4.0,36.0,4,0,10000.0,13000.0,0.08,3000.0,30.00,-4.0,26.0,3,0,10000.000000,13000.000000,0.080000,3000.000000,30.000000,0.000000,31.000000,3.500000,0.000000,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,